<a href="https://colab.research.google.com/github/itsstephxc/BigData/blob/main/Copy_of_08_21_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
# Binary Classification Template (if you have a problem that requires you to pick from 2 outcomes)

In [101]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score
from time import time
from sklearn.metrics import explained_variance_score, mean_absolute_error, r2_score
from matplotlib import pyplot
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier, PassiveAggressiveClassifier, Perceptron
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from sklearn.svm import SVC, NuSVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

In [102]:
%time
# Set your environment in a notebook:
# Set the Environment
# Ignore Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#Write out the versions of all packages to requirements.txt
!pip freeze >> requirements.txt

# Remove the restriction on Jupyter that limits the columns displayed (the ... in the middle)
# pd.set_option('max_columns',None)
pd.options.display.max_columns = None
# pd.options.display.max_rows = None
# Docs: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.set_option.html#

# Pretty Display of variables.  for instance, you can call df.head() and df.tail() in the same cell and BOTH display w/o print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# List of ALL Magic Commands.  To run a magic command %var  --- i.e.:  %env
%lsmagic
# %env  -- list environment variables
# %%time  -- gives you information about how long a cel took to run
# %%timeit -- runs a cell 100,000 times and then gives you the average time the cell will take to run (can be LONG)
# %pdb -- python debugger

# to display nice model diagram
from sklearn import set_config
set_config(display='diagram')

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

print("\n Numpy: " + np.__version__)
print("\n sklearn: " + sklearn.__version__)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 6.2 µs


Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %conda  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %shell  %store  %sx  %system  %tb  %tensorflow_version  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%bigquery  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%late


 Numpy: 1.23.5

 sklearn: 1.2.2


In [112]:
df = pd.read_csv('https://raw.githubusercontent.com/tidyverse/dplyr/main/data-raw/starwars.csv')
df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,sex,gender,homeworld,species,films,vehicles,starships
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19.0,male,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...","Snowspeeder, Imperial Speeder Bike","X-wing, Imperial shuttle"
1,C-3PO,167.0,75.0,NaN,gold,yellow,112.0,none,masculine,Tatooine,Droid,"The Empire Strikes Back, Attack of the Clones,...",NaN,NaN
2,R2-D2,96.0,32.0,NaN,"white, blue",red,33.0,none,masculine,Naboo,Droid,"The Empire Strikes Back, Attack of the Clones,...",NaN,NaN
3,Darth Vader,202.0,136.0,none,white,yellow,41.9,male,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...",NaN,TIE Advanced x1
4,Leia Organa,150.0,49.0,brown,light,brown,19.0,female,feminine,Alderaan,Human,"The Empire Strikes Back, Revenge of the Sith, ...",Imperial Speeder Bike,NaN


In [113]:
# Things to look out for;	a) no missing values and b) only numeric values (encoding)


In [114]:
# Calculating the number of missing values in each column
missing_values = df.isnull().sum()

# Calculating the percentages of missing values for each column
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_values, missing_percentages

(name           0
 height         6
 mass          28
 hair_color     5
 skin_color     0
 eye_color      0
 birth_year    44
 sex            4
 gender         4
 homeworld     10
 species        4
 films          0
 vehicles      76
 starships     67
 dtype: int64,
 name           0.000000
 height         6.896552
 mass          32.183908
 hair_color     5.747126
 skin_color     0.000000
 eye_color      0.000000
 birth_year    50.574713
 sex            4.597701
 gender         4.597701
 homeworld     11.494253
 species        4.597701
 films          0.000000
 vehicles      87.356322
 starships     77.011494
 dtype: float64)

In [115]:
# Checking the columns in the dataframe
columns_in_data = df.columns
columns_in_data


Index(['name', 'height', 'mass', 'hair_color', 'skin_color', 'eye_color',
       'birth_year', 'sex', 'gender', 'homeworld', 'species', 'films',
       'vehicles', 'starships'],
      dtype='object')

In [116]:
df.fillna(0, inplace=True)

print(df.nunique())

name          87
height        46
mass          39
hair_color    13
skin_color    31
eye_color     15
birth_year    37
sex            5
gender         3
homeworld     49
species       38
films         24
vehicles      11
starships     17
dtype: int64


In [117]:
# Calculating the number of missing values in each column
missing_values = df.isnull().sum()

# Calculating the percentages of missing values for each column
missing_percentages = (df.isnull().sum() / len(df)) * 100

missing_values, missing_percentages

(name          0
 height        0
 mass          0
 hair_color    0
 skin_color    0
 eye_color     0
 birth_year    0
 sex           0
 gender        0
 homeworld     0
 species       0
 films         0
 vehicles      0
 starships     0
 dtype: int64,
 name          0.0
 height        0.0
 mass          0.0
 hair_color    0.0
 skin_color    0.0
 eye_color     0.0
 birth_year    0.0
 sex           0.0
 gender        0.0
 homeworld     0.0
 species       0.0
 films         0.0
 vehicles      0.0
 starships     0.0
 dtype: float64)

In [118]:
df.dtypes
# Object is most likely categorical and a string. Int/Float are numeric or binary. You want to know what they data types are. We want to encode the data if categorical.

name           object
height        float64
mass          float64
hair_color     object
skin_color     object
eye_color      object
birth_year    float64
sex            object
gender         object
homeworld      object
species        object
films          object
vehicles       object
starships      object
dtype: object

In [119]:
# Filter out rows where 'sex' is 'none'
df = df[df['sex'] != 'none']

# Map 'male' and 'female' in the 'sex' column to 1 and 0, respectively
df['sex'] = df['sex'].map({'male': 1, 'female': 0})

# Identifying the categorical columns in the dataset
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()
if 'name' in categorical_columns:
    categorical_columns.remove('name')  # Removing 'name' column

# One-hot encoding for the remaining categorical columns (excluding 'sex' since we've already handled it)
encoded_data = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Ensure all columns are in a numerical format
for col in encoded_data.columns:
    encoded_data[col] = pd.to_numeric(encoded_data[col], errors='coerce')

encoded_data.head()

<ipython-input-119-58b1b12417ed>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sex'] = df['sex'].map({'male': 1, 'female': 0})


,name,height,mass,birth_year,sex,hair_color_auburn,"hair_color_auburn, grey","hair_color_auburn, white",hair_color_black,hair_color_blond,hair_color_blonde,hair_color_brown,"hair_color_brown, grey",hair_color_grey,hair_color_none,hair_color_unknown,hair_color_white,"skin_color_blue, grey",skin_color_brown,skin_color_brown mottle,"skin_color_brown, white",skin_color_dark,skin_color_fair,"skin_color_fair, green, yellow",skin_color_green,"skin_color_green, grey","skin_color_green-tan, brown",skin_color_grey,"skin_color_grey, blue","skin_color_grey, green, yellow","skin_color_grey, red",skin_color_light,skin_color_mottled green,skin_color_orange,skin_color_pale,skin_color_red,"skin_color_red, blue, white",skin_color_tan,skin_color_unknown,skin_color_white,"skin_color_white, blue",skin_color_yellow,eye_color_blue,eye_color_blue-gray,eye_color_brown,eye_color_dark,eye_color_gold,"eye_color_green, yellow",eye_color_hazel,eye_color_orange,eye_color_pink,eye_color_red,eye_color_unknown,eye_color_white,eye_color_yellow,gender_feminine,gender_masculine,homeworld_Alderaan,homeworld_Aleen Minor,homeworld_Bespin,homeworld_Bestine IV,homeworld_Cato Neimoidia,homeworld_Cerea,homeworld_Champala,homeworld_Chandrila,homeworld_Concord Dawn,homeworld_Corellia,homeworld_Coruscant,homeworld_Dathomir,homeworld_Dorin,homeworld_Endor,homeworld_Eriadu,homeworld_Geonosis,homeworld_Glee Anselm,homeworld_Haruun Kal,homeworld_Iktotch,homeworld_Iridonia,homeworld_Kalee,homeworld_Kamino,homeworld_Kashyyyk,homeworld_Malastare,homeworld_Mirial,homeworld_Mon Cala,homeworld_Muunilinst,homeworld_Naboo,homeworld_Nal Hutta,homeworld_Ojom,homeworld_Quermia,homeworld_Rodia,homeworld_Ryloth,homeworld_Serenno,homeworld_Shili,homeworld_Skako,homeworld_Socorro,homeworld_Stewjon,homeworld_Sullust,homeworld_Tatooine,homeworld_Toydaria,homeworld_Trandosha,homeworld_Troiken,homeworld_Tund,homeworld_Umbara,homeworld_Utapau,homeworld_Vulpter,homeworld_Zolan,species_Aleena,species_Besalisk,species_Cerean,species_Chagrian,species_Clawdite,species_Dug,species_Ewok,species_Geonosian,species_Gungan,species_Human,species_Hutt,species_Iktotchi,species_Kaleesh,species_Kaminoan,species_Kel Dor,species_Mirialan,species_Mon Calamari,species_Muun,species_Nautolan,species_Neimodian,species_Pau'an,species_Quermian,species_Rodian,species_Skakoan,species_Sullustan,species_Tholothian,species_Togruta,species_Toong,species_Toydarian,species_Trandoshan,species_Twi'lek,species_Vulptereen,species_Wookiee,species_Xexto,species_Yoda's species,species_Zabrak,films_Attack of the Clones,"films_Attack of the Clones, Revenge of the Sith","films_Attack of the Clones, Revenge of the Sith, A New Hope","films_Attack of the Clones, The Phantom Menace","films_Attack of the Clones, The Phantom Menace, Revenge of the Sith",films_Return of the Jedi,"films_Return of the Jedi, The Force Awakens",films_Revenge of the Sith,"films_Revenge of the Sith, A New Hope",films_The Empire Strikes Back,"films_The Empire Strikes Back, Attack of the Clones, Return of the Jedi","films_The Empire Strikes Back, Attack of the Clones, The Phantom Menace, Revenge of the Sith, Return of the Jedi","films_The Empire Strikes Back, Attack of the Clones, The Phantom Menace, Revenge of the Sith, Return of the Jedi, A New Hope","films_The Empire Strikes Back, Return of the Jedi","films_The Empire Strikes Back, Return of the Jedi, A New Hope","films_The Empire Strikes Back, Return of the Jedi, A New Hope, The Force Awakens","films_The Empire Strikes Back, Revenge of the Sith, Return of the Jedi, A New Hope","films_The Empire Strikes Back, Revenge of the Sith, Return of the Jedi, A New Hope, The Force Awakens",films_The Force Awakens,films_The Phantom Menace,"films_The Phantom Menace, Return of the Jedi, A New Hope","films_The Phantom Menace, Revenge of the Sith",vehicles_AT-ST,vehicles_Flitknot speeder,vehicles_Imperial Speeder Bike,vehicles_Koro-2 Exodrive airspeeder,vehicles_Sith speeder,vehicles_Snowspeeder,"vehicles_Snowspeeder, Imperial Spee

In [120]:
df.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,sex,gender,homeworld,species,films,vehicles,starships
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19.0,1.0,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...","Snowspeeder, Imperial Speeder Bike","X-wing, Imperial shuttle"
3,Darth Vader,202.0,136.0,none,white,yellow,41.9,1.0,masculine,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...",0,TIE Advanced x1
4,Leia Organa,150.0,49.0,brown,light,brown,19.0,0.0,feminine,Alderaan,Human,"The Empire Strikes Back, Revenge of the Sith, ...",Imperial Speeder Bike,0
5,Owen Lars,178.0,120.0,"brown, grey",light,blue,52.0,1.0,masculine,Tatooine,Human,"Attack of the Clones, Revenge of the Sith, A N...",0,0
6,Beru Whitesun lars,165.0,75.0,brown,light,blue,47.0,0.0,feminine,Tatooine,Human,"Attack of the Clones, Revenge of the Sith, A N...",0,0


In [121]:
X = df.drop(['gender'], axis=1)
y = df['gender']
X.head()

,name,height,mass,hair_color,skin_color,eye_color,birth_year,sex,homeworld,species,films,vehicles,starships
0,Luke Skywalker,172.0,77.0,blond,fair,blue,19.0,1.0,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...","Snowspeeder, Imperial Speeder Bike","X-wing, Imperial shuttle"
3,Darth Vader,202.0,136.0,none,white,yellow,41.9,1.0,Tatooine,Human,"The Empire Strikes Back, Revenge of the Sith, ...",0,TIE Advanced x1
4,Leia Organa,150.0,49.0,brown,light,brown,19.0,0.0,Alderaan,Human,"The Empire Strikes Back, Revenge of the Sith, ...",Imperial Speeder Bike,0
5,Owen Lars,178.0,120.0,"brown, grey",light,blue,52.0,1.0,Tatooine,Human,"Attack of the Clones, Revenge of the Sith, A N...",0,0
6,Beru Whitesun lars,165.0,75.0,brown,light,blue,47.0,0.0,Tatooine,Human,"Attack of the Clones, Revenge of the Sith, A N...",0,0


In [122]:
# ENCODE THE REST OF YOUR FEATURES
X = pd.get_dummies(X)
X.head()

,height,mass,birth_year,sex,name_Ackbar,name_Adi Gallia,name_Anakin Skywalker,name_Arvel Crynyd,name_Ayla Secura,name_Bail Prestor Organa,name_Barriss Offee,name_Ben Quadinaros,name_Beru Whitesun lars,name_Bib Fortuna,name_Biggs Darklighter,name_Boba Fett,name_Bossk,name_Captain Phasma,name_Chewbacca,name_Cliegg Lars,name_Cordé,name_Darth Maul,name_Darth Vader,name_Dexter Jettster,name_Dooku,name_Dormé,name_Dud Bolt,name_Eeth Koth,name_Finis Valorum,name_Finn,name_Gasgano,name_Greedo,name_Gregar Typho,name_Grievous,name_Han Solo,name_Jabba Desilijic Tiure,name_Jango Fett,name_Jar Jar Binks,name_Jek Tono Porkins,name_Jocasta Nu,name_Ki-Adi-Mundi,name_Kit Fisto,name_Lama Su,name_Lando Calrissian,name_Leia Organa,name_Lobot,name_Luke Skywalker,name_Luminara Unduli,name_Mace Windu,name_Mas Amedda,name_Mon Mothma,name_Nien Nunb,name_Nute Gunray,name_Obi-Wan Kenobi,name_Owen Lars,name_Padmé Amidala,name_Palpatine,name_Plo Koon,name_Poe Dameron,name_Poggle the Lesser,name_Quarsh Panaka,name_Qui-Gon Jinn,name_Ratts Tyerell,name_Raymus Antilles,name_Rey,name_Ric Olié,name_Roos Tarpals,name_Rugor Nass,name_Saesee Tiin,name_San Hill,name_Sebulba,name_Shaak Ti,name_Shmi Skywalker,name_Sly Moore,name_Tarfful,name_Taun We,name_Tion Medon,name_Wat Tambor,name_Watto,name_Wedge Antilles,name_Wicket Systri Warrick,name_Wilhuff Tarkin,name_Yarael Poof,name_Yoda,name_Zam Wesell,hair_color_0,hair_color_auburn,"hair_color_auburn, grey","hair_color_auburn, white",hair_color_black,hair_color_blond,hair_color_blonde,hair_color_brown,"hair_color_brown, grey",hair_color_grey,hair_color_none,hair_color_unknown,hair_color_white,skin_color_blue,"skin_color_blue, grey",skin_color_brown,skin_color_brown mottle,"skin_color_brown, white",skin_color_dark,skin_color_fair,"skin_color_fair, green, yellow",skin_color_green,"skin_color_green, grey","skin_color_green-tan, brown",skin_color_grey,"skin_color_grey, blue","skin_color_grey, green, yellow","skin_color_grey, red",skin_color_light,skin_color_mottled green,skin_color_orange,skin_color_pale,skin_color_red,"skin_color_red, blue, white",skin_color_tan,skin_color_unknown,skin_color_white,"skin_color_white, blue",skin_color_yellow,eye_color_black,eye_color_blue,eye_color_blue-gray,eye_color_brown,eye_color_dark,eye_color_gold,"eye_color_green, yellow",eye_color_hazel,eye_color_orange,eye_color_pink,eye_color_red,eye_color_unknown,eye_color_white,eye_color_yellow,homeworld_0,homeworld_Alderaan,homeworld_Aleen Minor,homeworld_Bespin,homeworld_Bestine IV,homeworld_Cato Neimoidia,homeworld_Cerea,homeworld_Champala,homeworld_Chandrila,homeworld_Concord Dawn,homeworld_Corellia,homeworld_Coruscant,homeworld_Dathomir,homeworld_Dorin,homeworld_Endor,homeworld_Eriadu,homeworld_Geonosis,homeworld_Glee Anselm,homeworld_Haruun Kal,homeworld_Iktotch,homeworld_Iridonia,homeworld_Kalee,homeworld_Kamino,homeworld_Kashyyyk,homeworld_Malastare,homeworld_Mirial,homeworld_Mon Cala,homeworld_Muunilinst,homeworld_Naboo,homeworld_Nal Hutta,homeworld_Ojom,homeworld_Quermia,homeworld_Rodia,homeworld_Ryloth,homeworld_Serenno,homeworld_Shili,homeworld_Skako,homeworld_Socorro,homeworld_Stewjon,homeworld_Sullust,homeworld_Tatooine,homeworld_Toydaria,homeworld_Trandosha,homeworld_Troiken,homeworld_Tund,homeworld_Umbara,homeworld_Utapau,homeworld_Vulpter,homeworld_Zolan,species_0,species_Aleena,species_Besalisk,species_Cerean,species_Chagrian,species_Clawdite,species_Dug,species_Ewok,species_Geonosian,species_Gungan,species_Human,species_Hutt,species_Iktotchi,species_Kaleesh,species_Kaminoan,species_Kel Dor,species_Mirialan,species_Mon Calamari,species_Muun,species_Nautolan,species_Neimodian,species_Pau'an,species_Quermian,species_Rodian,species_Skakoan,species_Sullustan,species_Tholothian,species_Togruta,species_Toong,species_Toydarian,species_Trandoshan,species_Twi'lek,species_Vulptereen,species_Wookiee,species_Xexto,species_Yoda's species,species_Zabrak,films_A New Hope,films_Attack of the Clones,"films_Attack of the Clones, Revenge of the Si

In [130]:
# Quick Model Training and Evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [131]:
X_train = X_train.dropna()
y_train = y_train.loc[X_train.index]


In [132]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear')))
models.append(('SGD', SGDClassifier()))
models.append(('Ridge', RidgeClassifier()))
models.append(('PAC', PassiveAggressiveClassifier()))
models.append(('Perceptron', Perceptron()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('QDA', QuadraticDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('NearestCentroid', NearestCentroid()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('ExtraTree', ExtraTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('BNB', BernoulliNB()))
models.append(('ComplementNB', ComplementNB()))
models.append(('MultinomialNB', MultinomialNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('NuSVC', NuSVC()))
models.append(('LinearSVC', LinearSVC()))
models.append(('RF', RandomForestClassifier()))
models.append(('GB', GradientBoostingClassifier()))
models.append(('ExtraTrees', ExtraTreesClassifier()))
models.append(('Bagging', BaggingClassifier()))
models.append(('AdaBoost', AdaBoostClassifier()))
models.append(('MLP', MLPClassifier()))
models.append(('XGBoost', XGBClassifier(use_label_encoder=False, eval_metric='logloss')))

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [133]:
model = NuSVC(nu=0.5)  # or any other value between 0 and 1


In [134]:
# Scoring metrics:
# 'accuracy', 'balanced_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'f1_micro', 'f1_macro', 'f1_weighted',
# 'neg_log_loss', 'precision', 'recall', 'jaccard', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted'
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    start = time()
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)
    model.fit(X_train, y_train)
    train_time = time() - start
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    predict_time = time() - start
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
    print("Score for each of the 10 K-fold tests: ",cv_results)
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print()

fig = pyplot.figure(figsize=(15, 10))  # Change the numbers as needed
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
pyplot.boxplot(results)
ax.set_xticklabels(names, rotation = 45)  # Added rotation for better visibility
pyplot.show()

LogisticRegression(solver='liblinear')

LR: 0.902381 (0.109343)
Score for each of the 10 K-fold tests:  [0.85714286 0.83333333 0.66666667 1.         1.         1.
 0.83333333 1.         0.83333333 1.        ]
LogisticRegression(solver='liblinear')
	Training time: 0.010s
	Prediction time: 0.078s



SGDClassifier()

SGD: 0.719048 (0.184980)
Score for each of the 10 K-fold tests:  [0.85714286 0.33333333 0.83333333 0.66666667 0.83333333 1.
 0.66666667 0.5        0.66666667 0.83333333]
SGDClassifier()
	Training time: 0.010s
	Prediction time: 0.092s



RidgeClassifier()

...

MultinomialNB()

MultinomialNB: 0.735714 (0.154469)
Score for each of the 10 K-fold tests:  [0.85714286 0.66666667 0.83333333 0.5        0.83333333 1.
 0.83333333 0.5        0.66666667 0.66666667]
MultinomialNB()
	Training time: 0.012s
	Prediction time: 0.089s



SVC(gamma='auto')

SVM: 0.885714 (0.149375)
Score for each of the 10 K-fold tests:  [0.85714286 1.         1.         0.83333333 1.         1.
 0.83333333 0.83333333 0.5        1.        ]
SVC(gamma='auto')
	Training time: 0.009s
	Prediction time: 0.080s



ValueError: ignored